In [ ]:
import sys
sys.path.append('../src/')

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cmocean as cmo
import xarray as xr

from laddie import Laddie
from geometry import Geometry
from forcing import Forcing
from tools import im,jm

%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2

In [ ]:
geom = Geometry('../../data/isomip/Ocean3_input_geom_v1.01.nc')
geom.coarsen(N=2)
geom = geom.create()

forc = Forcing(geom).isomip('WARM')

#Create laddie
laddie = Laddie(forc)

laddie.Ah = 6 #Horizontal viscosity
laddie.Kh = 1 #Horizontal diffusivity

laddie.Cdtop = 1.1e-3 #Top drag coefficient
laddie.minD = 6.8 #Minimum layer thickness
laddie.dt = 320 #Time step
laddie.nu = .3

laddie.saveday = 10 #Interval at which to save output
laddie.restday = 10 #Interval at which to save restart file and restart run

In [ ]:
ds = laddie.compute(days=30)

In [ ]:
fig,ax = plt.subplots(3,1,figsize=(15,7),sharex=True)

im = ax[0].pcolormesh(laddie.x,laddie.y,(im(laddie.U[1,:,:])**2+jm(laddie.V[1,:,:])**2)**.5,shading='auto',cmap='cmo.amp')
ax[0].set_title('Speed [m/s]')
plt.colorbar(im,ax=ax[0])

im = ax[1].pcolormesh(laddie.x,laddie.y,np.where(laddie.tmask==1,laddie.zb,np.nan),shading='auto',cmap='cmo.deep_r')
plt.colorbar(im,ax=ax[1])
ax[1].set_title('ice shelf draft [m]')

im = ax[2].pcolormesh(laddie.x,laddie.y,np.where(laddie.tmask==1,3600*24*365.25*laddie.melt,np.nan),shading='auto',cmap='inferno',norm=mpl.colors.LogNorm(vmin=1,vmax=100))
plt.colorbar(im,ax=ax[2])
ax[2].set_title('melt rate [m/yr]')

for n in [0,1,2]:
    ax[n].set_aspect('equal', 'box')